In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

### os
import os
import sys

### datetimes
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

### scipy
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import cartopy.crs as ccrs
import dask
from dask.diagnostics import ProgressBar
from tqdm import tqdm

### plotting
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns

In [3]:
import pathlib

HOME = pathlib.Path.home()
CWD = pathlib.Path.cwd()

### import local functions for the processing of the C3S forecasts 

In [4]:
sys.path.append('../..')

In [5]:
from ICU_Water_Watch import C3S, domains, geo, utils

### parameters for papermill

In [6]:
# Parameters
country = "Marshall Islands"
admin = "Northern Marshall Islands"
lag = 0
quantiles = "tercile"


### list of GCMs 

In [7]:
list_GCMs = ['ECMWF','UKMO','METEO_FRANCE','CMCC','DWD', 'NCEP', 'JMA', 'ECCC_CanCM4i', 'ECCC_GEMO_NEMO']

### get today's date 

In [8]:
date = datetime.utcnow()

### apply lag 

In [9]:
date = date - relativedelta(months=lag)

In [10]:
print(f"will process forecasts issued in {date:%B %Y}")

will process forecasts issued in November 2021


In [11]:
year = date.year 

In [12]:
year 

2021

In [13]:
month = date.month 

In [14]:
month

11

### list files for the individual GCMs

In [15]:
opath = CWD.parents[1].joinpath("outputs/C3S")

In [16]:
opath = opath.joinpath(f'admin/{quantiles}')

In [17]:
lfiles = list(opath.glob(f"{quantiles}_probs_forecasts_from_{year}_{str(month).zfill(2)}_{utils.sanitize_name(country)}_{utils.sanitize_name(admin)}_*.csv"))

In [18]:
len(lfiles)

9

In [19]:
df_all = []
for fname in lfiles: 
    df = pd.read_csv(fname, header=[0,1], index_col=[0,1,2,3])
    df_all.append(df)

In [20]:
df = pd.concat(df_all, axis=0)

In [21]:
# step = 3
# df = df.xs(str(step), axis=1, level=0)

In [22]:
df

3  \
tercile                                                                      1   
2021-11-01 Marshall Islands Northern Marshall Islands CMCC           38.000000   
                                                      NCEP           56.451613   
                                                      ECCC_GEM_NEMO   0.000000   
                                                      ECMWF          35.294118   
                                                      ECCC_CanCM4i   20.000000   
                                                      DWD            26.000000   
                                                      UKMO           16.129032   
                                                      JMA            20.512821   
                                                      METEO_FRANCE   11.764706   

                                                                                \
tercile                                                                      2   
2021-11-01 Marshall Islands Northern Marshall Islands CMCC           38.000000   
                                                      NCEP           32.258065   
                                                      ECCC_GEM_NEMO   0.000000   
                                                      ECMWF          29.411765   
                                                      ECCC_CanCM4i   10.000000   
                                                      DWD            44.000000   
                                                      UKMO           53.225806   
                                                      JMA            50.000000   
                                                      METEO_FRANCE   29.411765   

                                                                                 \
tercile                                                                       3   
2021-11-01 Marshall Islands Northern Marshall Islands CMCC            24.000000   
                                                      NCEP            11.290323   
                                                      ECCC_GEM_NEMO  100.000000   
                                                      ECMWF           35.294118   
                                                      ECCC_CanCM4i    70.000000   
                                                      DWD             30.000000   
                                                      UKMO            30.645161   
                                                      JMA             29.487179   
                                                      METEO_FRANCE    58.823529   

                                                                             4  \
tercile                                                                      1   
2021-11-01 Marshall Islands Northern Marshall Islands CMCC           42.000000   
                                                      NCEP           25.000000   
                                                      ECCC_GEM_NEMO   0.000000   
                                                      ECMWF          19.607843   
                                                      ECCC_CanCM4i    0.000000   
                                                      DWD            12.000000   
                                                      UKMO           11.290323   
                                                      JMA            11.538462   
                                                      METEO_FRANCE   11.764706   

                                                                                \
tercile                                                                      2   
2021-11-01 Marshall Islands Northern Marshall Islands CMCC           22.000000   
                                                      NCEP           25.000000   
                                                      ECCC_GEM_NEMO   0.000000   
                                            

In [23]:
df.index[0]

('2021-11-01', 'Marshall Islands', 'Northern Marshall Islands', 'CMCC')

In [24]:
tuple(list(df.index[-1][:-1]) + ['MME'])

('2021-11-01', 'Marshall Islands', 'Northern Marshall Islands', 'MME')

In [25]:
df.loc[tuple(list(df.index[-1][:-1]) + ['MME']),:] = df.mean(0)

In [26]:
df

3  \
tercile                                                                      1   
2021-11-01 Marshall Islands Northern Marshall Islands CMCC           38.000000   
                                                      NCEP           56.451613   
                                                      ECCC_GEM_NEMO   0.000000   
                                                      ECMWF          35.294118   
                                                      ECCC_CanCM4i   20.000000   
                                                      DWD            26.000000   
                                                      UKMO           16.129032   
                                                      JMA            20.512821   
                                                      METEO_FRANCE   11.764706   
                                                      MME            24.905810   

                                                                                \
tercile                                                                      2   
2021-11-01 Marshall Islands Northern Marshall Islands CMCC           38.000000   
                                                      NCEP           32.258065   
                                                      ECCC_GEM_NEMO   0.000000   
                                                      ECMWF          29.411765   
                                                      ECCC_CanCM4i   10.000000   
                                                      DWD            44.000000   
                                                      UKMO           53.225806   
                                                      JMA            50.000000   
                                                      METEO_FRANCE   29.411765   
                                                      MME            31.811933   

                                                                                 \
tercile                                                                       3   
2021-11-01 Marshall Islands Northern Marshall Islands CMCC            24.000000   
                                                      NCEP            11.290323   
                                                      ECCC_GEM_NEMO  100.000000   
                                                      ECMWF           35.294118   
                                                      ECCC_CanCM4i    70.000000   
                                                      DWD             30.000000   
                                                      UKMO            30.645161   
                                                      JMA             29.487179   
                                                      METEO_FRANCE    58.823529   
                                                      MME             43.282257   

                                                                             4  \
tercile                                                                      1   
2021-11-01 Marshall Islands Northern Marshall Islands CMCC           42.000000   
                                                      NCEP           25.000000   
                                                      ECCC_GEM_NEMO   0.000000   
                                                      ECMWF          19.607843   
                                                      ECCC_CanCM4i    0.000000   
                                                      DWD            12.000000   
                                                      UKMO           11.290323   
                                                      JMA            11.538462   
                                                      METEO_FRANCE   11.764706   
                                                      MME            14.800148   

                                                                                \
tercile                                    

In [27]:
df_rounded = []
for step in df.columns.get_level_values(0).unique(): 
    df_step = df.xs(step, axis=1, level=0)
    df_step = df_step.T.apply(utils.round_to_100_percent, **{'digit_after_decimal':0}).T.astype(int)
    df_step.columns = pd.MultiIndex.from_product([[step], df_step.columns])
    df_rounded.append(df_step)

In [28]:
df_rounded = pd.concat(df_rounded, axis=1)

In [29]:
opath

PosixPath('/home/nicolasf/operational/ICU/development/hotspots/code/ICU_Water_Watch/outputs/C3S/admin/tercile')

In [30]:
df_rounded.to_csv(opath.joinpath(f"MME/{quantiles}_probs_forecasts_from_{year}_{str(month).zfill(2)}_{utils.sanitize_name(country)}_{utils.sanitize_name(admin)}_MME.csv"))

In [31]:
df_rounded

3      \
tercile                                                               1   2   
2021-11-01 Marshall Islands Northern Marshall Islands CMCC           38  38   
                                                      NCEP           57  32   
                                                      ECCC_GEM_NEMO   0   0   
                                                      ECMWF          35  30   
                                                      ECCC_CanCM4i   20  10   
                                                      DWD            26  44   
                                                      UKMO           16  53   
                                                      JMA            21  50   
                                                      METEO_FRANCE   12  29   
                                                      MME            25  32   

                                                                           4  \
tercile                                                                3   1   
2021-11-01 Marshall Islands Northern Marshall Islands CMCC            24  42   
                                                      NCEP            11  25   
                                                      ECCC_GEM_NEMO  100   0   
                                                      ECMWF           35  19   
                                                      ECCC_CanCM4i    70   0   
                                                      DWD             30  12   
                                                      UKMO            31  11   
                                                      JMA             29  12   
                                                      METEO_FRANCE    59  12   
                                                      MME             43  15   

                                                                              \
tercile                                                               2    3   
2021-11-01 Marshall Islands Northern Marshall Islands CMCC           22   36   
                                                      NCEP           25   50   
                                                      ECCC_GEM_NEMO   0  100   
                                                      ECMWF          18   63   
                                                      ECCC_CanCM4i   10   90   
                                                      DWD            34   54   
                                                      UKMO           39   50   
                                                      JMA            41   47   
                                                      METEO_FRANCE   27   61   
                                                      MME            24   61   

                                                                      5      \
tercile                                                               1   2   
2021-11-01 Marshall Islands Northern Marshall Islands CMCC           34  30   
                                                      NCEP            2  26   
                                                      ECCC_GEM_NEMO   0   0   
                                                      ECMWF           8  23   
                                                      ECCC_CanCM4i   10  10   
                                                      DWD            10  22   
                                                      UKMO           19  49   
                                                      JMA             9  23   
                                                      METEO_FRANCE    6  23   
                                                      MME            11  23   

                                                                          
tercile                                                                3  
2021-11-01 Marshall Islands Northern Marshall Islands CMCC            36  
                         